# UD1 – Bloques 7 e 8: Exercicios con Delta Lake e Apache Iceberg 

Este notebook recolle exercicios guiados para repetir os exemplos dos apuntamentos:

- 07.Delta-Lake
- 08.Apache-iceberg

Empregaremos o dataset `penguins.csv` (por exemplo, o clásico *Palmer Penguins*), gardado en:

```text
/home/hadoop/work/data/penguins.csv
```
Hai que subilo a HDFS, por exemplo a `/data`
Cada exercicio puntuará a puntuación indicada se está correcto e 0 en caso contrario.

## 0. Preparación da sesión de Spark

Se a sesión de Spark non está creada, inicialízaa nesta cela.
(Non puntúa, pero é necesaria para o resto).

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("BoletinLakehouse").getOrCreate()
spark.version

---

# Parte 1 – Delta Lake sobre HDFS

Nesta parte imos crear e manexar unha táboa Delta almacenada en HDFS
a partir do ficheiro `penguins.csv`.

### Exercicio 1 (0,5 puntos) – Ler o dataset `penguins.csv` nun DataFrame

Ler o ficheiro CSV con `header = True` e `inferSchema = True` e inspeccionar o esquema
e algunhas filas.

In [ ]:
penguins_csv_path = "/data/penguins.csv"



### Exercicio 2 (0,5 puntos) – Gardar o DataFrame como táboa Delta en HDFS

Garda o DataFrame `df` en formato Delta na ruta `/datalake/penguins_delta` (crea o directorio `datalake` en HDFS se non existe).

In [ ]:
delta_path_hdfs = "/datalake/penguins_delta"


Podes comprobar o contido en HDFS dende o contedor do namenode:

```bash
hdfs dfs -ls /datalake/penguins_delta
hdfs dfs -ls /datalake/penguins_delta/_delta_log
```

### Exercicio 3 (0,75 puntos) – Lectura e consulta da táboa Delta

Ler de novo a táboa Delta desde HDFS e facer unha consulta que devolva a media do `bill_length_mm` por especie.

### Exercicio 4 (0,75 puntos) – Actualización con MERGE INTO

Simula unha actualización de datos:
- crea un pequeno DataFrame con algunhas novas observacións (por exemplo, 
  unha especie existente cun `bill_length_mm` lixeiramente distinto, e outra observación nova).
- fai un `MERGE` contra a táboa Delta.

Pista: podes usar como clave de combinación algunha combinación de columnas
por exemplo (`species`, `island`). O obxectivo é practicar o MERGE,
non tanto a corrección biolóxica dos datos.

In [ ]:
from delta.tables import DeltaTable
from pyspark.sql import functions as F

delta_table = DeltaTable.forPath(spark, delta_path_hdfs)

novas_obs = [
    # posible actualización (mesma especie / illa)
    ("Adelie", "Torgersen", 40.0, 18.7),
    # posible rexistro novo
    ("Gentoo", "Biscoe", 50.1, 15.3),
]

df_novos = spark.createDataFrame(
    novas_obs,
    ["species", "island", "bill_length_mm", "bill_depth_mm"]
)

# Complementamos co resto de columnas como nulas se non existen
for col in df_delta.columns:
    if col not in df_novos.columns:
        df_novos = df_novos.withColumn(col, F.lit(None))

df_novos = df_novos.select(df_delta.columns)  # mesma orde/columnas

# Continuar aqui:



### Exercicio 5 (0,75 puntos) – Versionado e *time travel* en Delta Lake

1. Usa `DESCRIBE HISTORY` para ver o historial da táboa.
2. Le a versión inicial (por exemplo, `versionAsOf = 0`) e compáraa coa versión actual.

In [ ]:
# 1


In [ ]:
#2


---

# Parte 2 – Delta Lake sobre MinIO (S3A)

Nesta parte imos repetir unha operación similar, pero gardando os datos nun bucket de MinIO,
empregando o esquema `s3a://`.

### Exercicio 6 (0,5 puntos) – Gardar e ler a táboa Delta en MinIO

1. Garda o DataFrame `df`  en `s3a://delta-lake/penguins_delta` (recorda crear o bucket `delta-lake` se non existe).
2. Léeo de novo desde MinIO e comproba que os datos son accesibles.

In [ ]:
delta_path_s3 = "s3a://delta-lake/penguins_delta"



Podes verificar o contido tamén con:

```bash
hadoop fs -ls s3a://delta-lake/penguins_delta
```
e na consola web de MinIO.

---

# Parte 3 – Apache Iceberg sobre HDFS (catálogo `local`)

Agora imos usar Apache Iceberg sobre HDFS, empregando o catálogo `local`
configurado no clúster.

### Exercicio 7 (0,5 puntos) – Crear un namespace Iceberg para penguins

Crea un namespace chamado `penguins` no catálogo `local` e comproba que aparece listado.

### Exercicio 8 (0,75 puntos) – Crear unha táboa Iceberg a partir de `penguins.csv`

Usa o DataFrame `df` lido de `penguins.csv` para crear a táboa `local.penguins.observacions`
en formato Iceberg.

### Exercicio 9 (0,75 puntos) – Consultas SQL sobre a táboa Iceberg

Realiza as seguintes consultas:

- número de rexistros por especie
- media de `body_mass_g` por especie

### Exercicio 10 (0,75 puntos) – Evolución de esquema en Iceberg

Engade unha columna nova chamada `notas` de tipo STRING á táboa Iceberg
e comproba o novo esquema.

### Exercicio 11 (0,5 puntos) – Inserción de datos respectando os tipos

Insire algunha fila nova na táboa, tendo coidado co tipo das columnas.

No caso de que o teu `penguins.csv` non teña datas, podes inserir deixando
algunhas columnas a NULL ou adaptando á estrutura real do dataset.

### Exercicio 12 (0,75 puntos) – Time travel en Iceberg

1. Lista os snapshots dispoñibles na táboa `local.penguins.observacions`.
2. Elixe un `snapshot_id` e consulta o estado da táboa nese snapshot.

In [ ]:
# 1


In [ ]:
# 2



---

# Parte 4 – Apache Iceberg sobre MinIO (catálogo `minio`)

Por último, imos crear e manexar unha táboa Iceberg almacenada en MinIO,
empregando o catálogo `minio` configurado sobre `s3a://spark/iceberg-warehouse`.

### Exercicio 13 0,75 puntos) – Crear namespace en `minio`

Crea o namespace `penguins` no catálogo `minio`.

### Exercicio 14 (0,75 puntos) – Crear táboa Iceberg en MinIO a partir de `penguins.csv`

Crea a táboa `minio.penguins.observacions` usando o DataFrame `df`.

### Exercicio 15 (0,75 puntos) – Inspeccionar ficheiros físicos en MinIO

Lista os ficheiros coñecidos por Iceberg para esta táboa e observa como cambian
despois da reescritura/particionamento.